In [4]:
import pandas as pd
import numpy as np

import os
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)

# read tweets related data

In [6]:
tweets = pd.read_csv("all/tweets\\tweet_final.csv")
tweets.drop(columns=['Unnamed: 0'], inplace=True)
tweets.rename(columns={'date': 'timestamp'}, inplace=True)
tweets['date'] = pd.to_datetime(tweets.timestamp.map(lambda x: x[:10]))

# check if block and tweets have common dates

In [8]:
dfs[0]['date'] = pd.to_datetime(dfs[0]['date'])
common_dates = dfs[0]['date'].isin(tweets['date'])
matching_rows = dfs[0][common_dates]
len(matching_rows)

347

# data cleaning

In [10]:
tweets

,user_name,user_followers,timestamp,text,retweets,isRetweet,favorites,isFlagged,date
0,jack,5.0,2021-04-11 19:03:04,$KICK - It’s not fruit cake. \n\n$VET $BTC $ET...,-1.0,False,-1.0,f,2021-04-11
1,jack,5.0,2021-04-10 23:29:15,Be a part of history. Be a part of $KICK. \n...,-1.0,False,-1.0,f,2021-04-10
2,jack,5.0,2021-04-10 21:40:06,$KICK - 7 Days. Did you buy?\n\n$VET $BTC $ETH...,-1.0,False,-1.0,f,2021-04-10
3,jack,5.0,2021-04-10 21:39:33,$KICK - At this pace we can be at .001 before ...,-1.0,False,-1.0,f,2021-04-10
4,jack,5.0,2021-04-10 21:38:42,"$500 = 660,000 $KICK Tokens. Short term goal i...",-1.0,False,-1.0,f,2021-04-10
...,...,...,...,...,...,...,...,...,...
2577,elonmusk,219700000.0,2024-09-02 04:08:08,True. \n\nI was just with some friends who wer...,-1.0,False,-1.0,f,2024-09-02
2578,elonmusk,219700000.0,2024-09-02 06:26:25,RT @AlexandreFiles: Em uma decisão de 18 de ag...,-1.0,False,-1.0,f,2024-09-02
2579,elonmusk,219700000.0,2024-09-02 15:28:17,The free market works incredibly well,-1.0,False,-1.0,f,2024-09-02
2580,elonmusk,219700000.0,2024-09-02 16:07:50,President @JMilei is doing an incredible job r...,-1.0,False,-1.0,f,2024-09-02


In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.preprocessing import OneHotEncoder

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'(.)\1+', r'\1\1', text) #remove repeated characters
    text = re.sub(r'[^\w\s]', '', text) #remove special characters

    tokens = word_tokenize(text)

    stop_words = set(stopwords.words('english')) - {"no", "nor", "not"}
    filtered_tokens = [word for word in tokens if word not in stop_words]

    lemmer = nltk.WordNetLemmatizer()
    lem_tokens = [lemmer.lemmatize(token) for token in tokens]
    cleaned = ' '.join(lem_tokens)
    
    return cleaned

def one_hot_encoding(df, col):
    encoder = OneHotEncoder(sparse_output=False)
    
    labeled = encoder.fit_transform(df[col])
    
    df_with_labels = pd.DataFrame(labeled, columns=encoder.get_feature_names_out(col))
    print(df_with_labels.shape)
    df_combined = pd.concat([df, df_with_labels], axis=1)
    
    df_combined = df_combined.drop(col, axis=1)
    return df_combined

def sentiment(text, sid):
    compound = sid.polarity_scores(text)['compound']
    if compound > 0.05:
        return 'pos'
    elif compound < -0.05:
        return 'neg'
    else: 
        return 'neu'

def sentiment_analysis(df):
    sid = SentimentIntensityAnalyzer()
    df['sentiment'] = df.cleaned_tweets.map(lambda text : sentiment(text, sid))
    print(df.sentiment.value_counts())
    df_encoded =  one_hot_encoding(df, ['sentiment'])
    return df_encoded
    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\qiavo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qiavo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
tweets['cleaned_tweets'] = tweets.text.map(lambda tweet : preprocess_text(tweet))

onehot_df = one_hot_encoding(tweets, ['user_name'])
onehot_df
onehot_df = sentiment_analysis(onehot_df)

(2582, 45)
sentiment
pos    1510
neg     694
neu     378
Name: count, dtype: int64
(2582, 3)


In [13]:
onehot_df['isRetweet'] = onehot_df['isRetweet'].astype(int)
onehot_df['isFlagged'] = onehot_df['isFlagged'].replace({'t': 1, 'f': 0})
onehot_df.drop(columns=['text', 'cleaned_tweets', 'timestamp'], inplace=True)
onehot_df

,user_followers,retweets,isRetweet,favorites,isFlagged,date,user_name_APompliano,user_name_AriDavidPaul,user_name_AriannaSimpson,user_name_BarrySilbert,...,user_name_realDonaldTrump,user_name_rogerkver,user_name_saylor,user_name_stablekwon,user_name_starkness,user_name_twobitidiot,user_name_tyler,sentiment_neg,sentiment_neu,sentiment_pos
0,5.0,-1.0,0,-1.0,0,2021-04-11,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,5.0,-1.0,0,-1.0,0,2021-04-10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5.0,-1.0,0,-1.0,0,2021-04-10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,5.0,-1.0,0,-1.0,0,2021-04-10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5.0,-1.0,0,-1.0,0,2021-04-10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2577,219700000.0,-1.0,0,-1.0,0,2024-09-02,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2578,219700000.0,-1.0,0,-1.0,0,2024-09-02,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2579,219700000.0,-1.0,0,-1.0,0,2024-09-02,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2580,219700000.0,-1.0,0,-1.0,0,2024-09-02,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
onehot_df.to_csv("cleaned_tweets.csv")

In [10]:
tweet = pd.read_csv("all/tweets\\tweets_all2.csv")
tweet['date'] = pd.to_datetime(tweet['date'])
tweet.drop(columns=['Unnamed: 0'], inplace=True)
tweet

,user_name,user_followers,date,text,retweets,isRetweet,favorites,isFlagged
0,jack,5.0,2021-04-11 19:03:04,$KICK - It’s not fruit cake. \n\n$VET $BTC $ET...,-1.0,False,-1.0,f
1,jack,5.0,2021-04-10 23:29:15,Be a part of history. Be a part of $KICK. \n...,-1.0,False,-1.0,f
2,jack,5.0,2021-04-10 21:40:06,$KICK - 7 Days. Did you buy?\n\n$VET $BTC $ETH...,-1.0,False,-1.0,f
3,jack,5.0,2021-04-10 21:39:33,$KICK - At this pace we can be at .001 before ...,-1.0,False,-1.0,f
4,jack,5.0,2021-04-10 21:38:42,"$500 = 660,000 $KICK Tokens. Short term goal i...",-1.0,False,-1.0,f
...,...,...,...,...,...,...,...,...
9286,brian_armstrong,1554518.0,2021-08-04 22:10:45,6/ The infrastructure bill also imposes sweepi...,39.0,False,294.0,-1
9287,muneeb,122151.0,2021-08-05 11:57:26,Treasury for the city of Miami received $150K ...,23.0,False,170.0,-1
9288,jespow,223709.0,2021-08-06 20:53:24,RT @fightfortheftr: #Cryptocurrency supporters...,364.0,True,0.0,-1
9289,RaoulGMI,1200000.0,2021-08-11 16:18:06,RT @RealVisionBot: #bitcoin did overtake #Ethe...,18.0,True,0.0,-1


In [12]:
tweet['cleaned_tweets'] = tweet.text.map(lambda tweet : preprocess_text(tweet))

onehot_df = one_hot_encoding(tweet, ['user_name'])
onehot_df
onehot_df = sentiment_analysis(onehot_df)
onehot_df['isRetweet'] = onehot_df['isRetweet'].astype(int)
onehot_df['isFlagged'] = onehot_df['isFlagged'].replace({'t': 1, 'f': 0})
onehot_df.drop(columns=['text', 'cleaned_tweets'], inplace=True)
onehot_df

(9291, 80)
sentiment
pos    5251
neg    2077
neu    1963
Name: count, dtype: int64
(9291, 3)


,user_followers,date,retweets,isRetweet,favorites,isFlagged,user_name_APompliano,user_name_AndreCronjeTech,user_name_AriDavidPaul,user_name_AriannaSimpson,...,user_name_sammcingvale,user_name_saylor,user_name_shayne_coplan,user_name_stablekwon,user_name_starkness,user_name_twobitidiot,user_name_tyler,sentiment_neg,sentiment_neu,sentiment_pos
0,5.0,2021-04-11 19:03:04,-1.0,0,-1.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,5.0,2021-04-10 23:29:15,-1.0,0,-1.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5.0,2021-04-10 21:40:06,-1.0,0,-1.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,5.0,2021-04-10 21:39:33,-1.0,0,-1.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5.0,2021-04-10 21:38:42,-1.0,0,-1.0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9286,1554518.0,2021-08-04 22:10:45,39.0,0,294.0,-1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9287,122151.0,2021-08-05 11:57:26,23.0,0,170.0,-1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9288,223709.0,2021-08-06 20:53:24,364.0,1,0.0,-1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9289,1200000.0,2021-08-11 16:18:06,18.0,1,0.0,-1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [14]:
onehot_df.to_csv("cleaned_tweets2.csv")